<a href="https://colab.research.google.com/github/Melanee-Melanee/OCR/blob/main/easyocr_custum_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JaidedAI/EasyOCR.git

fatal: destination path 'EasyOCR' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#!pip install easyocr

In [2]:
%cd /content/EasyOCR/trainer

/content/EasyOCR/trainer


In [4]:
!pwd

/content/EasyOCR/trainer


In [6]:
!ls

all_data      craft	  model.py  __pycache__  saved_models  trainer.ipynb  utils.py
config_files  dataset.py  modules   README.md	 test.py       train.py


In [7]:
!unzip /content/EasyOCR/trainer/all_data/en_train_filtered.zip

unzip:  cannot find or open /content/EasyOCR/trainer/all_data/en_train_filtered.zip, /content/EasyOCR/trainer/all_data/en_train_filtered.zip.zip or /content/EasyOCR/trainer/all_data/en_train_filtered.zip.ZIP.


In [8]:
!unzip /content/EasyOCR/trainer/all_data/en_val.zip

unzip:  cannot find or open /content/EasyOCR/trainer/all_data/en_val.zip, /content/EasyOCR/trainer/all_data/en_val.zip.zip or /content/EasyOCR/trainer/all_data/en_val.zip.ZIP.


In [14]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [15]:
cudnn.benchmark = True
cudnn.deterministic = False

In [16]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [18]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: en_train_filtered
all_data/en_train_filtered
sub-directory:	/en_train_filtered	 num samples: 882
num total samples of en_train_filtered: 882 x 1.0 (total_data_usage_ratio) = 882
num samples of en_train_filtered per batch: 32 x 1.0 (batch_ratio) = 32


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 882
--------------------------------------------------------------------------------
No Transformation module specified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model input parameters 64 600 20 1 256 256 97 34 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
        (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
        (10): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
        (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


NameError: name 'validation' is not defined

In [ ]:
!cp /root/EasyOCR//user_network/custom_example.yaml /root/.EasyOCR//user_network/custom_example.yaml

In [ ]:
!cp /root/EasyOCR/user_network/custom_example.py /root/.EasyOCR/user_network/custom_example.py

In [ ]:
!cp /root/EasyOCR/model/custom_example.pth /root/.EasyOCR/model/custom_example.pth

In [ ]:
import easyocr
reader = easyocr.Reader(['en'], recog_network='custom_example')
result = reader.readtext('/content/999.jpg', detail=0)
print(result)